# Compute sea level changes due to atmospheric pressure ("inverted barometer" or "IB correction")

**Datasets this notebook expects to find** under the below given datapaths:
- `land_mask_0p25_oceanFull_invGrd.nc` (land-ocean mask by Bernd Uebbing)

Files that are **created** in this notebook (in case they don't exist):
- `era5_mean_sea_level_pressure.nc` (Global monthly mean sea level pressure for the period 01/1992 - 12/2021, downloaded from ERA5)
- `ib_correction_era5_1992-2021.nc`

In [1]:
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import datetime

In [2]:
main_datapath_input = '/home/bene/PhD-docs/80_papers/1_dataset_combination/data/input/'
main_datapath_output = '/home/bene/PhD-docs/80_papers/1_dataset_combination/data/output/'

In [3]:
g = 9.80665 # mean gravitational acceleration [m/s^2]
d = 1027 # standard value for ocean surface water density [kg/m^3]

#### Download ERA5 global monthly mean sea level pressure for the period 01/1992 - 12/2021

In [4]:
if not os.path.isfile(main_datapath_input+'era5_mean_sea_level_pressure.nc'):
    import cdsapi

    c = cdsapi.Client()

    c.retrieve(
        'reanalysis-era5-single-levels-monthly-means',
        {
            'product_type': 'monthly_averaged_reanalysis',
            'variable': 'mean_sea_level_pressure',
            'year': [
                '1992', '1993', '1994',
                '1995', '1996', '1997',
                '1998', '1999', '2000',
                '2001', '2002', '2003',
                '2004', '2005', '2006',
                '2007', '2008', '2009',
                '2010', '2011', '2012',
                '2013', '2014', '2015',
                '2016', '2017', '2018',
                '2019', '2020', '2021',
            ],
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'time': '00:00',
            'format': 'netcdf',
        },
        main_datapath_input + 'era5_mean_sea_level_pressure.nc')

#### Compute and save monthly IB correction

In [5]:
ds = xr.open_dataset(main_datapath_input + 'era5_mean_sea_level_pressure.nc', engine='netcdf4')

ocean mask:

In [6]:
mask = xr.open_dataset(main_datapath_input + 'land_mask_0p25_oceanFull_invGrd.nc', engine='netcdf4')
mask = mask.squeeze('time').reset_coords('time', drop=True)
mask = mask.z > 0.5
mask = mask[:,:1440]
mask = np.flipud(mask)
mask_temp1 = mask[:,:720]
mask_temp2 = mask[:,720:]
mask = np.hstack((mask_temp2, mask_temp1))
mask = np.invert(mask)
mask = np.broadcast_to(mask, ds.msl.shape)
msl_ocean = np.ma.masked_array(ds.msl, mask=mask)

latitude weighting & mean over total ocean surface:

In [ ]:
weights = np.cos(ds.latitude * np.pi / 180)
weights = np.resize(weights, (len(ds.longitude),len(ds.latitude)))
weights = np.transpose(weights)
weights = np.ma.masked_array(weights, mask=mask[0])

msl_ocean_w = msl_ocean * weights
msl_ocean_w_mean = msl_ocean_w.sum(axis=(1,2)) / weights.sum()

IB correction:

In [ ]:
msl_ocean_w_mean = msl_ocean_w_mean.reshape((len(msl_ocean_w_mean),1,1))
msl_ocean_w_mean = np.broadcast_to(msl_ocean_w_mean, (msl_ocean.shape))
corr = (msl_ocean - msl_ocean_w_mean) / (d * g)

In [ ]:
plt.figure(1, figsize=(20,10))
plt.matshow(corr.mean(axis=0), fignum=1)
plt.colorbar()
plt.title('IB correction in [m]')

In [ ]:
data_vars = {'IB_correction':(['time', 'lat', 'lon'], corr,
            {'long_name':'Inverted barometer height correction',
            'units': 'm',
            'pressure data':'ERA5 monthly mean sea level pressure',
            'period': '01/1992 - 12/2021',
            })}

coords = {'time': (['time'], ds.time.values),
        'lat': (['lat'], ds.latitude.values),
        'lon': (['lon'], ds.longitude.values)}

attrs = {'description':'Inverted barometer correction for sea level heights from ERA5, globally.',
        'creation_date':json.dumps(datetime.now(), indent=4, sort_keys=True, default=str),
        'author':'Bene Aschenneller',
        'email':'s.aschenneller@utwente.nl'}

ds_corr = xr.Dataset(data_vars, coords, attrs)
ds_corr.to_netcdf(main_datapath_output + 'ib_correction_era5_1992-2021.nc', format='NETCDF4', encoding={'IB_correction':{'dtype':'int32', 'scale_factor': 1e-5}} )